In [1]:
import pandas
import csv
from pandas.core.frame import DataFrame
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/livialilli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/livialilli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Step 1

In [3]:
rentals_file = pandas.read_csv(("files/Airbnb_Texas_Rentals.csv"), sep =",", delimiter= None, header = "infer", names = None, index_col = None,
                       encoding="ISO-8859-1")

In [4]:
rentals_file.head(3)

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW â...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...


# Step 2

In [5]:
#creating tsv files
i = 0
for r in range(len(rentals_file)):
    record = rentals_file.loc[[r]]
    name = "doc_" + str(i) +".tsv"
    record.to_csv(path_or_buf = name, sep='\t')
    
    i += 1
    

KeyboardInterrupt: 

In [5]:
#function to read the tsv file
def read(file_name):
    read_file = pandas.read_csv(file_name, sep ="\t", delimiter= None, header = "infer", names = None, index_col = None, usecols = None,
                       encoding="ISO-8859-1")
    return read_file
    

In [6]:
#for now I just consider one tsv file
d = read("files/doc_0.tsv")

In [7]:
d

,Unnamed: 0,Unnamed: 0.1,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...


In [8]:
d["description"]

0    Welcome to stay in private room with queen bed...
Name: description, dtype: object

In [9]:
s = d["description"][0]

In [10]:
s

'Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private bedroom with sofa bed is available for additional guests. 10$ for an additional guest.\\n10min from IAH airport\\nAirport pick-up/drop off is available for $10/trip.'

In [11]:
col_names = ["average_rate_per_night","bedrooms_count", "city", "date_of_listing", "description", "latitude", "longitude","title","url"]

# Step 3

## Preprocessing

In [12]:
def stopWords(column_file):
    
    stopWords = set(stopwords.words('english'))
    words = word_tokenize(str(column_file))
    wordsFiltered = []

    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered
    

In [13]:
def punctuation(list):
    exclude = set(string.punctuation)
    for el in list:
        if el in exclude:
            list.remove(el)
    return list

In [14]:
def stemming(list):
    ps = PorterStemmer()
    output = []
    for word in list:
        stem_word = ps.stem(word)
        output.append(stem_word)
    return output
#it returns a list (of the current column) with all the stem-words


In [15]:
#example of application
s = stopWords(d["title"][0])
p = punctuation(s)
print(stemming(p))

['2', 'privat', 'rooms/bathroom', '10min', 'iah', 'airport']


In [16]:
#On ALL the COLUMNS
#applyng stopWords,punctuation and stemming functions for every column of the dataframe
#it returns a dictionary, where keys are the columns names, and for every key there is the list of cleaned words.
def all_col(read_file):
    dic = {}
    for name in col_names:
        s = str(read_file[name][0]).replace("\\n", " ")
        l = stopWords(s)
        l = punctuation(l)
        result = stemming(l)
        dic[name] = result
        #print(result)
    return dic

In [17]:
print(all_col(d))


{'average_rate_per_night': ['27'], 'bedrooms_count': ['2'], 'city': ['humbl'], 'date_of_listing': ['may', '2016'], 'description': ['welcom', 'stay', 'privat', 'room', 'queen', 'bed', 'detach', 'privat', 'bathroom', 'second', 'floor', 'anoth', 'privat', 'bedroom', 'sofa', 'bed', 'avail', 'addit', 'guest', '10', 'addit', 'guest', '10min', 'iah', 'airport', 'airport', 'pick-up/drop', 'avail', '10/trip'], 'latitude': ['30.0201379199512'], 'longitude': ['-95.29399600425128'], 'title': ['2', 'privat', 'rooms/bathroom', '10min', 'iah', 'airport'], 'url': ['http', '//www.airbnb.com/rooms/18520444', 'location=cleveland', '2C', '20tx']}


## 3.1 Conjunctive query

In [18]:
#we have to consider just description and title columns


def filter_keys(dictionary):
    keys = ["description", "title"]
    return {x: dictionary[x] for x in dictionary if x in keys}



### 3.1.1 Create your index

In [58]:
#for every tsv file, I apply the preprocess, I filter the columns that I need, and then I build the vocabulary
vocabulary = {}
for i in range(len(rentals_file)):
    name = "files/doc_" + str(i) +".tsv"
    df = read(name)
    clean_d = all_col(df)
    filtered_d = filter_keys(clean_d)
    values = filtered_d.values()
    for l in values:
        for el in l:
            if el not in vocabulary.keys():
                vocabulary[el] = [name]
            else:
                vocabulary[el].append(name)
    
    
#it has got as keys the words, as values all the docs where words are in 
    

In [59]:
keys = vocabulary.keys()
list_keys = list(keys)
file_voc = {}

for i in range(len(list_keys)):
    term = "term_id_" + str(i)
    
    file_voc[term] = list_keys[i]

#it has as keys the id terms, as values, the corrispondent words

In [61]:
file_voc_keys = list(file_voc.keys())

In [63]:
#this as as keys the term id, as values the docs where id's word is in
new = {}
for i in range(len(file_voc_keys)):
    new_key = file_voc_keys[i]
    old_key = file_voc[new_key]
    new[new_key] = vocabulary[old_key]

### 3.1.2 Execute the query

In [86]:
query = input()

welcome world


In [87]:
query = stopWords(query)

In [88]:
query = punctuation(query)

In [89]:
query = stemming(query)

In [90]:
query

['welcom', 'world']

In [105]:
docs = {}
for word in query:
    try:
        l = vocabulary[word]
        docs[word] = l
    except:
        pass

In [106]:
print(docs)

{'welcom': ['files/doc_0.tsv', 'files/doc_4.tsv', 'files/doc_22.tsv', 'files/doc_22.tsv', 'files/doc_50.tsv', 'files/doc_58.tsv', 'files/doc_58.tsv', 'files/doc_70.tsv', 'files/doc_70.tsv', 'files/doc_94.tsv', 'files/doc_104.tsv', 'files/doc_125.tsv', 'files/doc_146.tsv', 'files/doc_150.tsv', 'files/doc_196.tsv', 'files/doc_199.tsv', 'files/doc_200.tsv', 'files/doc_222.tsv', 'files/doc_236.tsv', 'files/doc_250.tsv', 'files/doc_275.tsv', 'files/doc_288.tsv', 'files/doc_296.tsv', 'files/doc_306.tsv', 'files/doc_332.tsv', 'files/doc_342.tsv', 'files/doc_346.tsv', 'files/doc_384.tsv', 'files/doc_404.tsv', 'files/doc_429.tsv', 'files/doc_435.tsv', 'files/doc_439.tsv', 'files/doc_443.tsv', 'files/doc_449.tsv', 'files/doc_463.tsv', 'files/doc_470.tsv', 'files/doc_474.tsv', 'files/doc_475.tsv', 'files/doc_500.tsv', 'files/doc_503.tsv', 'files/doc_504.tsv', 'files/doc_540.tsv', 'files/doc_549.tsv', 'files/doc_567.tsv', 'files/doc_589.tsv', 'files/doc_607.tsv', 'files/doc_610.tsv', 'files/doc_63

In [119]:
v = docs.values()

In [124]:
v = list(v)

In [126]:
intersection = set(v[0]).intersection(*v)

In [145]:
from IPython.display import display, HTML

for file in intersection:
    file = read(file)
    
    result = file[["title","description","city","url"]]
    display(result.style)
    #print(result.to_html())

,title,description,city,url
0,Kick Back Cabin- Country Townhome,"THE PERFECT GETAWAY SPOT! Why? It's the closest vacation rental within walking distance to the World Famous Market Days. Private, fenced in yard (pets welcome) surrounded by beautiful oak trees. BBQ, chiminea, and S'more fixins!! :)",Wimberley,https://www.airbnb.com/rooms/8833656?location=Colorado%20River%2C%20TX


,title,description,city,url
0,"Perfect home for work, play, or BMT graduation","Welcome to San Antonio where you will stay at this beautiful retreat. Filled with natural light, modern, open space, and a wonderful outdoor entertaining area. Located 6 miles from Sea World, 11 miles from Lackland AFB, and only 14 miles to downtown SA. This home is perfect for business, vacation, Basic Military Training (BMT) graduation where you can spend much derserved time with your new Airman. Looking forward to seeing you!",San Antonio,https://www.airbnb.com/rooms/16154135?location=Bulverde%2C%20TX


,title,description,city,url
0,LOOK NO FURTHER! This IS YOUR BMT Destination!,"Welcome to San Antonio! When you stay with us, you are minutes from Lackland AFB, Sea World, shopping, dining, entertainment, downtown San Antonio & the famous River Walk. The home has an open floor plan, ample seating & a large master bedroom with garden tub. A deck in the backyard is a great place to relax. DVD PLAYER, SMART TV, Wii with kid friendly games.",San Antonio,https://www.airbnb.com/rooms/18491165?location=Castroville%2C%20TX


,title,description,city,url
0,"COMFORTABLE, QUIET & PRIVATE HOME CENTRAL LOCATION","1 upstairs bedroom with 2 twin beds. Private bathroom next to bedroom. Free WiFi. Quiet neighborhood close to airport, River Walk, downtown, the Alamo, museums, the Zoo, shopping, Fiesta Texas, Sea World and great restaurants. We warmly welcome you with morning coffee/tea. Feel free to use the well-equipped kitchen and laundry facilities. Relax on the patio. This is your home-away-from home!! Your privacy is our priority. You will be comfortable here!",San Antonio,https://www.airbnb.com/rooms/17668828?location=Alamo%20Heights%2C%20TX


,title,description,city,url
0,"Silvia & Luiso ""home sweet home""","2 Bedroom and 1 bathroom, One queen bed on each Bedroom(perfect for a family of 4 due to we only have 1 full bathroom).small children's are welcome!!!!!!!!\n\nSAFE And CLEAN !!!!!!! brand new quiet family home nice bathroom between rooms and large closets. T.V.(Both rooms) cable and roku ( gets Netflix,Hulu,etc.) . \n\nFREE WI-FI !!!!!!!!\n\n\nNOT A PARTY HOUSE . \n\nAll races ,religions,nationalities WELCOMED!!!!!!. \n\nNO PETS . NO EXTRA PERSON ALLOWED.\n\nOk to use microwave,Keurig coffe maker,assigned area in fridge ( store your leftovers from restaurants ). Our house is located in LACKLAND A.F.B. AREA (7.5 miles ,approx 20 min) 3 miles from 410 interstate loop a,Hwy 90 and Loop 1604.\n\nAPPROXIMATE DISTANCES FROM OUR HOUSE TO : SAT !! San Antonio Intl, airport(18 Miles- 26 Min.) Sea world (5.2 Miles - 15 Min.), Downtown River walk ( 15 Miles - 27 Min.), Fiesta Texas Six Flags ( 21 Miles- 29 Min.) Medical center San Antonio (14 Miles-24 Min),The Shops at LA CANTERA (22 Miles- 32 Min.),North Star Mall ( 16 Miles-24 Min.),IngramPark Mall (7.4 Miles-15Min.),San Marcos Outlets(55 Miles-60Min.), Austin !Texas (91 Miles-1hr 32 min).\n\n\nFREE COFFEE,TEA,APPLES,BANANAS,FRUIT BARS,BOTTLED WATER.\n\n\nclose to HEB and Walmart (grocery stores ), Walgreens and CVS ( pharmacy) ,Gas stations( Valero,texaco,she'll etc).)IHOP and many Restaurants.\nThis house great for vacationers because not a party house !!!!! It's quiet for sleeping day or night.\n\nGUEST ACCESS\nfor the price per night you have access to 2 Bedrooms and 1 full Bathroom for up to 4 people maximum ( No extra people allowed) 2 persons per Bedroom with queen size bed each ( located up stairs , next to the family room ) , kitchen and dinning room and driveway (one car only).",San Antonio,https://www.airbnb.com/rooms/18999165?location=Castroville%2C%20TX


...

,title,description,city,url
0,The Haven-Cozy & Private,"The Haven is a cozy apartment with it's own courtyard. It has a full kitchen, with everything needed to enjoy meals at home. The living area has seating for four. The bathroom has a jetted tub and a full size washer and dryer. Well behaved pets are welcome. There's lot do do here, and we are 40 minutes from the Worlds Largest Casino, 20 minutes from Texas Motor Speedway, and about 45 minutes from Dallas or Fort Worth. Texas Woman's University and University of North Texas are very near.",Denton,https://www.airbnb.com/rooms/17960608?location=Argyle%2C%20TX


,title,description,city,url
0,GALLERY on the Blanco in Wimberley,"Unique home, pristine setting, incredible community. Recently remodeled with special attention to the use of materials that are energy efficient, non toxic, and aesthetically pleasing. The focus on conservation is evident throughout with hand crafted counters of recycled glass and resin, reclaimed wood floors and trim, volcanic limestone and resin tub, and more. The incorporation of antiques in the cabinets, stained glass windows, and individually selected doors create a timeless appeal. The recycled theme is carried through to the art, with a stunning school of fish individually sculpted out of aluminum cans, and the custom furniture which includes a dining room table made from a 150 year old mission door. \n\nWhile there is much to appreciate in the interior construction, it's the view that captures you. The home sits on a limestone bluff directly over the river with water views from almost every room in the house. The water is deep enough to paddle board and is perfect for tubing when the river is flowing. You can rest your lounge chair in the shallow water on the banks and relax to the soothing sounds of the rapids when you're through with your swim. \n\nHard to imagine a place this serene can also be convenient, but the location is unbeatable. We're a mile from town, Blue Hole Regional Park, and Wimberley's renown outdoor theater, The Corral. Jacob's Well is just a few minutes away. Fiber Arts enthusiasts will love Old Oaks Ranch. Live music and is always available in town, at the Whitewater Ampitheater or other nearby locations. In addition to all there is to do here, Wimberley is the perfect location to stay while you plan day trips to outlying areas. Head out on a hill country wine or barbecue tour, visit Hamilton Pool, tube the Guadalupe in New Braunfels, take in Gruene and the oldest dance hall in Texas, tour LBJ's boyhood home in Johnson City, or spend the day at America's largest water park, Schlitterbahn. Wimberley is home to many artists and musicians and there are opportunities to attend a variety of shows year round. San Antonio is only an hour away with The Alamo, 6 Flags, the zoo and more, or head 45 minutes north to Austin, the Live Music Capitol of the World!\n\nThe well stocked kitchen with professional cookware, gas stove, and every imaginable appliance will delight any cook. The enormous master suite is divine with it's cathedral ceiling, inviting decor and a bathroom to rival any spa. The architecture features high ceilings throughout with a 30 foot ceiling in the living room wrapped in wood. \n\nThere are no carpets, drapes or indoor plants to aggravate allergies. However, pets are on the property.\n\nThere is a greenhouse with an aquaponic system. You're welcome to partake of the current selection of organic produce.\n\nWe love this community and will gladly help tailor your stay to include your interests. No doubt, you will be enchanted, too.\n\nThe Blanco River, fire pit, horseshoe pit, a few tubes for the river, and admission to any barn concerts if they coincide with your stay.\n\nAs much or as little as you prefer.\n\nThe location is EXTREMELY convenient but tranquil. We have the river behind the house and ranches across the street. The grocery store is 1.5 miles away, and the Coral Outdoor Theatre is just down the road. Flite Acres is truly one of the most desirable streets in Wimberley.\n\nThere is no public transit. But you can rent horses down the road!",Wimberley,https://www.airbnb.com/rooms/558177?location=Canyon%20Lake%2C%20TX


,title,description,city,url
0,Huge!Luxurious! Family fun getaway near Sea World.,"Great location! Great space ! Welcome to our home , over 4,000sqft. Of things to do. Close to so many attractions, restraunts and shopping centers.\nThis home has a king size bed in the master bedroom, 3 queen size beds in the 3 other bedrooms and Two air mattresses!",San Antonio,https://www.airbnb.com/rooms/17317540?location=Castroville%2C%20TX


,title,description,city,url
0,Private Room & Bathroom in Quiet Neighborhood,"I was a host a few years back. I really enjoyed hosting people from all over the world. I also have stayed in many other host's homes. I love the AirBnB experience! \n\nYou are welcome to wash some clothes, or use the iron if you need to. You also are welcome to cook and eat here. \n \nI travel a lot, but if I am home, I've always got coffee, herbal teas, and fresh organic food. If I have just made a fresh batch of organic spinach walnut pesto, I will share it with you!",Austin,https://www.airbnb.com/rooms/17678915?location=Cedar%20Park%2C%20TX


,title,description,city,url
0,Kick Back Cabin- Country Townhome,"THE PERFECT GETAWAY SPOT! Why? It's the closest vacation rental within walking distance to the World Famous Market Days. Private, fenced in yard (pets welcome) surrounded by beautiful oak trees. BBQ, chiminea, and S'more fixins!! :)",Wimberley,https://www.airbnb.com/rooms/8833656?location=Canyon%20Lake%2C%20TX


## 3.2 Conjunctive query & Ranking score